In [45]:
import rosbag
from wpiutil.log import *
import struct

fname = "Log_24-04-04_22-03-36.wpilog"
wpilog = DataLogReader("/mnt/c/Users/matth/Downloads/" + fname)

In [54]:
from dataclasses import dataclass
from photonlibpy.photonPipelineResult import PhotonPipelineResult
from photonlibpy.packet import Packet


@dataclass
class SwerveModuleState:
    speedMps: float
    angleRad: float

def decodeStates(raw: bytes): 
    schema = "<dddddddd"
    state = struct.unpack(schema, raw)
    return [SwerveModuleState(state[i], state[i+1]) for i in range(0, 8, 2)]

def decodePacket(raw: bytes): 
    if len(raw) < 1:
        return None

    result = PhotonPipelineResult().populateFromPacket(Packet(raw))
    print(result)
    return result


topics = {
    # "/RealOutputs/Swerve/Achieved": ["getRaw", decodeStates],
    "/RealOutputs/Photon/Raw Camera Data 3": ["getRaw", decodePacket]
}

topicMap = {}

topicData = {}

for msg in wpilog: 
    if msg.isStart():
        start = msg.getStartData()
        if start.name in topics.keys():
            topicMap[start.entry] = start.name
        continue

    if msg.isControl():
        continue

    if msg.isSetMetadata():
        continue

    if msg.isSetMetadata():
        continue

    if msg.isFinish():
        topicMap.pop(msg.getFinishEntry())
        continue

    if msg.getEntry() in topicMap:

        topicName = topicMap[msg.getEntry()]
        
        if topicName not in topicData:
            topicData[topicName] = []
        
        method = topics[topicName][0]
        transformer = topics[topicName][1]
        topicData[topicName].append(transformer(getattr(msg, method)()))


In [51]:
topicData.keys()
topicData["/RealOutputs/Photon/Raw Camera Data 1"]

[None]

In [ ]:
bag = rosbag.Bag(fname + ".bag", 'w')

bag.write()